In [2]:
from moleculib.protein.dataset import ProteinDataset
from moleculib.protein.batch import PadBatch
from preprocess import StandardizeTransform
from torch.utils.data import DataLoader
from visualize import backbone_to_pdb
from en_denoiser import EnDenoiser
from einops import rearrange
import torch
import torch.nn.functional as F

# Load data

In [18]:
CHECKPOINT = "/Users/ido/genomator/bindiff/lightning_logs/checkpoints/version_2/checkpoints/epoch=999-step=2000.ckpt"
TRAIN_DIR = "data/cath_sanity/train"
transform = [StandardizeTransform()]
train_dataset = ProteinDataset(TRAIN_DIR, transform=transform, preload=True)

/Users/ido/genomator/venv/lib/python3.9/site-packages/biotite/structure/io/pdb/file.py:423: UserWarning: 862 elements were guessed from atom_name.
  warn("{} elements were guessed from atom_name.".format(rep_num))
/Users/ido/genomator/venv/lib/python3.9/site-packages/biotite/structure/io/pdb/file.py:423: UserWarning: 1188 elements were guessed from atom_name.
  warn("{} elements were guessed from atom_name.".format(rep_num))
/Users/ido/genomator/venv/lib/python3.9/site-packages/biotite/structure/io/pdb/file.py:423: UserWarning: 651 elements were guessed from atom_name.
  warn("{} elements were guessed from atom_name.".format(rep_num))
/Users/ido/genomator/venv/lib/python3.9/site-packages/biotite/structure/io/pdb/file.py:423: UserWarning: 1994 elements were guessed from atom_name.
  warn("{} elements were guessed from atom_name.".format(rep_num))
/Users/ido/genomator/venv/lib/python3.9/site-packages/biotite/structure/io/pdb/file.py:423: UserWarning: 1758 elements were guessed from atom_

In [19]:
loader = DataLoader(train_dataset, collate_fn=PadBatch.collate, batch_size=1, shuffle=False)
batch = next(iter(loader))

In [20]:
model = EnDenoiser.load_from_checkpoint(CHECKPOINT)
model.eval()

EnDenoiser(
  (transformer): EnTransformer(
    (token_emb): Embedding(23, 32)
    (time_mlp): Sequential(
      (0): SinusoidalPositionEmbeddings()
      (1): Linear(in_features=32, out_features=128, bias=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=128, out_features=128, bias=True)
    )
    (layers): ModuleList(
      (0): Block(
        (attn): Residual(
          (fn): EquivariantAttention(
            (time_mlp): Sequential(
              (0): SiLU()
              (1): Linear(in_features=128, out_features=64, bias=True)
            )
            (norm): LayerNorm()
            (to_qkv): Linear(in_features=32, out_features=768, bias=False)
            (to_out): Linear(in_features=256, out_features=32, bias=True)
            (coors_mlp): Sequential(
              (0): Linear(in_features=4, out_features=16, bias=False)
              (1): GELU(approximate='none')
              (2): Linear(in_features=16, out_features=4, bias=False)
            )
           

In [22]:
def rescale_protein(coords, std_const=9.0):
    return std_const * coords

In [23]:
def rearrange_coords(coords):
    new_coords = coords.squeeze(0)[:,:num_backbone, :]
    new_coords = rearrange(new_coords, "s b c -> (s b) c")
    return new_coords

# Visualize the sample backbone

In [24]:
pdb_fname = "test.pdb"
num_backbone = 4
coord = batch.atom_coord
coord = rescale_protein(rearrange_coords(coord))
seq = str(batch.sequence[0])
backbone_to_pdb(coord, seq, pdb_fname, num_backbone)

File test.pdb has been saved.


# Visualize noise steps

In [25]:
def visualize_step(batch, num_steps):
    ts = torch.tensor([num_steps])
    x = batch.atom_coord[:,:,:num_backbone, :]
    noised_x, noise = model.q_sample(x, ts)
    noised_x = coord = rescale_protein(rearrange_coords(noised_x))
    noised_fname = f"noised_{num_steps}.pdb"
    backbone_to_pdb(noised_x, seq, noised_fname, num_backbone)

In [26]:
num_timesteps = 100
for i in range(num_timesteps):
    visualize_step(batch, i)

File noised_0.pdb has been saved.
File noised_1.pdb has been saved.
File noised_2.pdb has been saved.
File noised_3.pdb has been saved.
File noised_4.pdb has been saved.
File noised_5.pdb has been saved.
File noised_6.pdb has been saved.
File noised_7.pdb has been saved.
File noised_8.pdb has been saved.
File noised_9.pdb has been saved.
File noised_10.pdb has been saved.
File noised_11.pdb has been saved.
File noised_12.pdb has been saved.
File noised_13.pdb has been saved.
File noised_14.pdb has been saved.
File noised_15.pdb has been saved.
File noised_16.pdb has been saved.
File noised_17.pdb has been saved.
File noised_18.pdb has been saved.
File noised_19.pdb has been saved.
File noised_20.pdb has been saved.
File noised_21.pdb has been saved.
File noised_22.pdb has been saved.
File noised_23.pdb has been saved.
File noised_24.pdb has been saved.
File noised_25.pdb has been saved.
File noised_26.pdb has been saved.
File noised_27.pdb has been saved.
File noised_28.pdb has been sa

# Perform backward diffusion

In [27]:
results = model.sample(batch, num_timesteps - 1)

sampling loop time step: 100it [00:19,  5.05it/s]                                                                                                           


In [30]:
[F.mse_loss(rearrange_coords(batch.atom_coord).unsqueeze(0), results[i]) for i in range(num_timesteps)]

[tensor(1.8852),
 tensor(1.9015),
 tensor(1.9293),
 tensor(1.9171),
 tensor(1.9359),
 tensor(1.9520),
 tensor(1.9681),
 tensor(1.9598),
 tensor(1.9722),
 tensor(1.9853),
 tensor(1.9954),
 tensor(1.9921),
 tensor(2.0098),
 tensor(1.9939),
 tensor(1.9974),
 tensor(2.0056),
 tensor(2.0251),
 tensor(2.0430),
 tensor(2.0648),
 tensor(2.0813),
 tensor(2.0918),
 tensor(2.1082),
 tensor(2.1118),
 tensor(2.1271),
 tensor(2.1406),
 tensor(2.1420),
 tensor(2.1482),
 tensor(2.1628),
 tensor(2.1668),
 tensor(2.1611),
 tensor(2.1480),
 tensor(2.1489),
 tensor(2.1569),
 tensor(2.1587),
 tensor(2.1624),
 tensor(2.1778),
 tensor(2.1930),
 tensor(2.1937),
 tensor(2.1995),
 tensor(2.2269),
 tensor(2.2293),
 tensor(2.2293),
 tensor(2.2255),
 tensor(2.2162),
 tensor(2.2166),
 tensor(2.2116),
 tensor(2.2177),
 tensor(2.2250),
 tensor(2.2370),
 tensor(2.2129),
 tensor(2.2179),
 tensor(2.2214),
 tensor(2.2123),
 tensor(2.2025),
 tensor(2.2026),
 tensor(2.1942),
 tensor(2.1960),
 tensor(2.2011),
 tensor(2.2040

In [ ]:
backward_coords = rescale_protein(results[0][0])

In [ ]:
backbone_to_pdb(backward_coords, seq, f"backward_{timesteps}.pdb", num_backbone)

# Manually Noise-Denoise

In [ ]:
coords, seq, masks = model.prepare_inputs(batch)
ts = torch.tensor([0])

# forward diffusion
noised_coords, noise = model.q_sample(coords, ts)
ts = ts.type(torch.float64)

# predict noisy input with transformer
feats, prediction = model.transformer(seq, noised_coords, ts, mask=masks)

In [ ]:
F.mse_loss(coords, noised_coords)

In [ ]:
seq_str = str(batch.sequence[0])
pred = rescale_protein(prediction[0])
backbone_to_pdb(pred, seq_str, "denoised.pdb", num_backbone)

In [ ]:
F.mse_loss(coords, prediction)

## Plotly

In [ ]:
import plotly.graph_objects as go

In [ ]:
def _scatter(name, ca_coord, color, visible=True):
    sc_coords = []
    for ca in zip(ca_coord, atom_mask):
        for atom in atoms[mask]:
            sc_coords.append(ca)
            sc_coords.append(atom)
            sc_coords.append([None, None, None])
    sc_coords = np.array(sc_coords)
    bb_x, bb_y, bb_z = ca_coord.T
    sc_x, sc_y, sc_z = sc_coords.T
    data = [
        go.Scatter3d(
            name=name + " coord",
            x=bb_x,
            y=bb_y,
            z=bb_z,
            marker=dict(
                size=7,
                colorscale="Viridis",
            ),
            line=dict(color=color, width=4),
            visible="legendonly" if not visible else True,
        ),
        go.Scatter3d(
            name=name + " vecs",
            x=sc_x,
            y=sc_y,
            z=sc_z,
            marker=dict(size=2, colorscale="Viridis"),
            line=dict(
                color=color,
                width=2,
            ),
            visible="legendonly",
        ),
    ]
    return data

In [ ]:
t = torch.tensor([num_steps])
x_start = coords
model.extract(model.sqrt_alphas_cumprod, t, x_start)

In [ ]:
F.mse_loss(coords, noised_coords)